In [52]:
from pathlib import Path
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import model_selection
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.model_selection import GridSearchCV, GroupKFold, cross_val_score, ShuffleSplit
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import SGDClassifier, HuberRegressor, SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer, PowerTransformer, PolynomialFeatures, KBinsDiscretizer
import numpy as np
from sklearn.svm import SVR
from sklearn import base
from collections import defaultdict
from sklearn.ensemble import RandomForestRegressor
# Standard plotly imports
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)


params = {'legend.fontsize': 20,
          'figure.figsize': (15, 10),
         'axes.labelsize': 20,
         'axes.titlesize': 20,
         'xtick.labelsize': 20,
         'ytick.labelsize': 20}
pylab.rcParams.update(params)
plt.close('all')

## Files are stored in the '2015' and '2016' subdirectories, relative to the working directory
path2015 = Path('../2015')
path2016 = Path('../2016')
dp = np.linspace(0,100,11)
dy = np.linspace(1945,2020,15)
pd.options.display.max_columns = None


In [ ]:
## Column names in the 2016 version of the Physician Compare National Downloadable File are simple and coding-friendly 
## except for a leading space in all columns after the first. That file is read first and its column names (with leading 
## space removed) used to replace column names of the 2015 version data after its file is read. Numerical columns are read 
## as floats (many have NaNs), except where that failed due to presence of a string. As of now, these are handled, when 
## necessary, on a case-by-case basis. ndf16.dtypes.equals(ndf15.dtypes) returns True.

#col_names16 = pd.read_csv(path2016 / 'Physician_Compare_National_Downloadable_File.csv', nrows=0).columns
#types_dict = {"NPI": float, " Ind_PAC_ID": float, " Grd_yr": float, " org_pac_id": float, " num_org_mem": float}
#types_dict.update({col: str for col in col_names16 if col not in types_dict})
ndf16 = pd.read_csv(path2016 / 'Physician_Compare_National_Downloadable_File.csv').rename(columns=lambda x: x.strip()).rename(columns=lambda x: x.lower())

#col_names16 = pd.read_csv(path2016 / 'Physician_Compare_2016_Individual_EP_Public_Reporting.csv', nrows=0).columns
#types_dict = {"NPI": float, " Ind_PAC_ID": float, " prf_rate": float, " patient_count": float}
#types_dict.update({col: str for col in col_names16 if col not in types_dict})
ips16 = pd.read_csv(path2016 / 'Physician_Compare_2016_Individual_EP_Public_Reporting.csv').rename(columns=lambda x: x.strip()).rename(columns=lambda x: x.lower())

census_regions = pd.read_csv('../us_census_bureau_regions_and_divisions.csv')
census_regions.rename(index=str, columns={'State Code':'st'}, inplace=True)
census_regions.rename(str.lower, axis='columns', inplace=True)

ndf16['org_pac_id'] = ndf16['org_pac_id'].astype(str)
ndf16['npi'] = ndf16['npi'].astype(str)
ndf16['hosp_afl_1'] = ndf16['hosp_afl_1'].astype(str)
ndf16['hosp_afl_2'] = ndf16['hosp_afl_2'].astype(str)
ndf16['hosp_afl_3'] = ndf16['hosp_afl_3'].astype(str)
ndf16['hosp_afl_4'] = ndf16['hosp_afl_4'].astype(str)
ndf16['hosp_afl_5'] = ndf16['hosp_afl_5'].astype(str)
ndf16['hosps']=(ndf16.hosp_afl_1+' '+ndf16.hosp_afl_2+' '+ndf16.hosp_afl_3+' '+ndf16.hosp_afl_4+' '+ndf16.hosp_afl_5).str.replace('\.0', '').str.replace('nan nan nan nan nan', '').str.replace(' nan', '')

# Add feature for the average number of members in the practitioner's organization(s).
ndf16['avg_num_org_mem'] = ndf16.groupby('npi')['num_org_mem'].transform('mean')
# Transform the prf_rate for measures with invs_msr == Y. This makes it so that 
# a numerically greater score is desirable for *all* measures.
#mdf.loc[mdf.invs_msr == 'Y', 'prf_rate'] = 100 - mdf.loc[mdf.invs_msr == 'Y', 'prf_rate']
# Add census Region and Division features to records based on state.
# https://github.com/cphalpert/census-regions/blob/master/us%20census%20bureau%20regions%20and%20divisions.csv
# Check for any with more than 1 state first
mdf['npi_num_st'] = mdf.groupby('npi')['st'].transform('nunique')
mdf['npi_num_zip'] = mdf.groupby('npi')['zip'].transform('nunique')
mdf['npi_num_measures'] = mdf.groupby('npi')['measure_id'].transform('nunique')

org_pac_ids = ndf16.groupby(['npi'])['org_pac_id'].unique().to_frame()
ndf16_org_pac_ids = pd.merge(ndf16, org_pac_ids, on='npi', how='left')


In [ ]:
ips_ndf16 = pd.merge(ips16, ndf16_org_pac_ids, on='npi', how='left')
mdf16 = pd.merge(ips_ndf16, census_regions, on='st', how='left')

# Replace NaN's in mhi, ehr, and assgn with 'N' for "No" to complement the 'Y' values 
values = {'mhi': 'N', 'ehr': 'N', 'assgn': 'N', 'sec_spec_1':'None', 'sec_spec_2':'None', 'sec_spec_3':'None', 'sec_spec_4':'None', 'hosp_afl_1':'None', 'hosp_afl_2':'None', 'hosp_afl_3':'None', 'hosp_afl_4':'None', 'hosp_afl_5':'None'}
mdf16 = mdf16.fillna(value=values)



# Feature engineering/manipulation


'''
>>>mdf16.columns

Index(['npi', 'ind_pac_id_x', 'lst_nm_x', 'frst_nm_x', 'measure_id',
       'measure_title', 'invs_msr', 'prf_rate', 'patient_count',
       'collection_type', 'live_site_ind', 'ind_pac_id_y', 'ind_enrl_id',
       'lst_nm_y', 'frst_nm_y', 'mid_nm', 'suff', 'gndr', 'cred', 'med_sch',
       'grd_yr', 'pri_spec', 'sec_spec_1', 'sec_spec_2', 'sec_spec_3',
       'sec_spec_4', 'sec_spec_all', 'org_nm', 'org_pac_id', 'num_org_mem',
       'adr_ln_1', 'adr_ln_2', 'ln_2_sprs', 'cty', 'st', 'zip', 'phn_numbr',
       'hosp_afl_1', 'hosp_afl_lbn_1', 'hosp_afl_2', 'hosp_afl_lbn_2',
       'hosp_afl_3', 'hosp_afl_lbn_3', 'hosp_afl_4', 'hosp_afl_lbn_4',
       'hosp_afl_5', 'hosp_afl_lbn_5', 'assgn', 'pqrs', 'ehr', 'mhi', 'state',
       'region', 'division'],
      dtype='object')

>>>print(mdf16.ind_pac_id_x.apply(str).describe())
>>>print(mdf16.npi.apply(str).describe())
                                        
count           685489
unique          126054
top       5294718938.0
freq               232                   One to one mapping between unique npi/ind_pac_id values. Using npi.
Name: ind_pac_id_x, dtype: object

count           685489
unique          126054
top       1861445538.0
freq               232
Name: npi, dtype: object

>>>print(mdf16.org_nm.apply(str).describe())
>>>print(mdf16.org_pac_id.apply(str).describe())

count     685489
unique     25618
top          nan
freq       63920
Name: org_nm, dtype: object

count     685489
unique     25760 <-- Most likely explanation is that a small fraction of org_nm values are repeated. Using org_pac_id.
top          nan
freq       63920
Name: org_pac_id, dtype: object

>>>print(mdf16.hosp_afl_1.apply(str).describe())
>>>print(mdf16.hosp_afl_lbn_1.apply(str).describe())

count     685489
unique      4494 <-- *Many* hospital names repeated, as expected. Using hosp_afl_1, etc. as unique categorical variables. 
top          nan
freq      185183
Name: hosp_afl_1, dtype: object

count     685489
unique      3437
top          nan
freq      185295
Name: hosp_afl_lbn_1, dtype: object

>>>print(mdf16.npi.apply(str).describe())
>>>mdf16[mdf16.groupby('npi')['st'].transform('nunique').ge(2)].pri_spec.describe()
>>>mdf16[mdf16.groupby('npi')['st'].transform('nunique').ge(2)].npi.apply(str).describe()

count           685489
unique          126054
top       1861445538.0
freq               232
Name: npi, dtype: object

count                    64836 <-- About 10% of total observations involve providers with an address in more than one state
unique                      68
top       DIAGNOSTIC RADIOLOGY
freq                     35041  
Name: pri_spec, dtype: object

count            64836
unique            5555 <-- Less than 5% of providers. For now, will filter our observations # of states for a provider > 1.
top       1639315260.0
freq               115
Name: npi, dtype: object

'''

# Numerical features worth keeping
# ['grd_yr', 'patient_count', 'avg_num_org_mem', 'npi_num_measures']

# Categorical features worth keeping (using measure_id rather than measure_title, since some titles were repeated).
# That is probably inconsequential since measure_id's with fewer than 100 observations are being filtered, 
# dropping the number considerably. ~84 to ~52?
# ['npi', 'measure_id', 'gndr', 'med_sch', 'pri_spec', 'sec_spec_1', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'org_nm', 'org_pac_id', 'hosp_afl_1',  'hosp_afl_2', 'hosp_afl_3', 'hosp_afl_4', 'hosp_afl_5', 'assgn', 'ehr', 'mhi', 'st', 'region', 'division']

mdf16.columns

Index(['npi', 'ind_pac_id_x', 'lst_nm_x', 'frst_nm_x', 'measure_id',
       'measure_title', 'invs_msr', 'prf_rate', 'patient_count',
       'collection_type', 'live_site_ind', 'ind_pac_id_y', 'ind_enrl_id',
       'lst_nm_y', 'frst_nm_y', 'mid_nm', 'suff', 'gndr', 'cred', 'med_sch',
       'grd_yr', 'pri_spec', 'sec_spec_1', 'sec_spec_2', 'sec_spec_3',
       'sec_spec_4', 'sec_spec_all', 'org_nm', 'org_pac_id', 'num_org_mem',
       'adr_ln_1', 'adr_ln_2', 'ln_2_sprs', 'cty', 'st', 'zip', 'phn_numbr',
       'hosp_afl_1', 'hosp_afl_lbn_1', 'hosp_afl_2', 'hosp_afl_lbn_2',
       'hosp_afl_3', 'hosp_afl_lbn_3', 'hosp_afl_4', 'hosp_afl_lbn_4',
       'hosp_afl_5', 'hosp_afl_lbn_5', 'assgn', 'pqrs', 'ehr', 'mhi', 'state',
       'region', 'division'],
      dtype='object')

print(mdf16.ind_pac_id_x.apply(str).describe())
print(mdf16.npi.apply(str).describe())
                                        
count           685489
unique          126054
top       5294718938.0
freq               232   One to one mapping between unique npi/ind_pac_id values. Using npi.
Name: ind_pac_id_x, dtype: object

count           685489
unique          126054
top       1861445538.0
freq               232
Name: npi, dtype: object

print(mdf16.org_nm.apply(str).describe())
print(mdf16.org_pac_id.apply(str).describe())

count     685489
unique     25618
top          nan
freq       63920
Name: org_nm, dtype: object

count     685489
unique     25760 <-- Most likely explanation is that a small fraction of org_nm values are repeated. Using org_pac_id.
top          nan
freq       63920
Name: org_pac_id, dtype: object

print(mdf16.hosp_afl_1.apply(str).describe())
print(mdf16.hosp_afl_lbn_1.apply(str).describe())

count     685489
unique      4494 <-- *Many* hospital names repeated, as expected. Using hosp_afl_1, etc. as unique categorical variables. 
top          nan
freq      185183
Name: hosp_afl_1, dtype: object

count     685489
unique      3437
top          nan
freq      185295
Name: hosp_afl_lbn_1, dtype: object

print(mdf16.npi.apply(str).describe())
print(mdf16[mdf16.groupby('npi')['st'].transform('nunique').ge(2)].pri_spec.describe())
print(mdf16[mdf16.groupby('npi')['st'].transform('nunique').ge(2)].npi.apply(str).describe())

count           685489
unique          126054
top       1861445538.0
freq               232
Name: npi, dtype: object

count                    64836 <-- About 10% of total observations involve providers with an address in more than one state
unique                      68
top       DIAGNOSTIC RADIOLOGY
freq                     35041  
Name: pri_spec, dtype: object

count            64836
unique            5555 <-- Less than 5% of providers. For now, will filter our observations # of states for a provider > 1.
top       1639315260.0
freq               115
Name: npi, dtype: object

'''

# Numerical features worth keeping
['grd_yr', 'patient_count', 'avg_num_org_mem', 'npi_num_measures']

# Categorical features worth keeping (using measure_id rather than measure_title, since some titles were repeated).
# That is probably inconsequential since measure_id's with fewer than 100 observations are being filtered, dropping the number considerably. ~84 to ~52?
['npi', 'measure_id', 'gndr', 'med_sch', 'pri_spec', 'sec_spec_1', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'org_nm', 'org_pac_id', 'hosp_afl_1',  'hosp_afl_2', 'hosp_afl_3', 'hosp_afl_4', 'hosp_afl_5', 'assgn', 'ehr', 'mhi', 'st', 'region', 'division']

In [ ]:
#print(mdf16.columns)
#mdf16.head(30)
#mdf16.loc[mdf16.org_nm == 'ANMED HEALTH']
#mdf16[mdf16.groupby('npi')['st'].transform('nunique').ge(2)].npi.apply(str).describe()
#temp = mdf16[['grd_yr', 'patient_count', 'npi', 'measure_id', 'gndr', 'med_sch', 'pri_spec', 'sec_spec_1', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'org_nm', 'org_pac_id', 'hosp_afl_1',  'hosp_afl_2', 'hosp_afl_3', 'hosp_afl_4', 'hosp_afl_5', 'assgn', 'ehr', 'mhi', 'st', 'region', 'division']]
#mdf16.pivot(index='npi', columns='org_nm')
#temp

mdf16
#ndf16[ndf16.groupby('npi')['org_pac_id'].transform('nunique').ge(18)]
#ndf16['org_pac_ids'] = 

# Want to get unique (org_nm, org_pac_id, num_org_mem)
# One provider with 18 different org_pac_id!
# Build 2 new columns, one with string representations of org_pac_id memberships, the other of hosp_afl_*'s
#

In [50]:
#pd.concat(ndf16.hosp_afl_1[0], ndf16.hosp_afl_2[0], ndf16.hosp_afl_3[0], ndf16.hosp_afl_4[0], ndf16.hosp_afl_5[0])
#ndf16.hosps[15] == ndf16.hosps[2031915]
ndf16.hosps
#type(ndf16.hosp_afl_1[0])

0                               490126 490048
1                               140010 140281
2                               140010 140281
3                               140010 140281
4                               140010 140281
5                               140010 140281
6                               140010 140281
7                               140010 140281
8                               140010 140281
9                               140010 140281
10                360112 360262 360081 360074
11                360112 360262 360081 360074
12                360112 360262 360081 360074
13                360112 360262 360081 360074
14                360112 360262 360081 360074
15                                           
16                                     360098
17                                      60024
18                                      60024
19                                     100072
20                                     100072
21                                

In [ ]:


mdf_trim = mdf_loc.iloc[:, np.r_[0, 4, 5, 7, 8, 9, 17, 19, 20, 21, 47, 49, 50, 51, 52, 57, 58, 59]].drop_duplicates()
# 

mdf_cln1 = mdf_fnl[['grd_yr', 'patient_count', 'avg_num_org_mem', 'npi_num_measures', 'measure_id', 'med_sch', 'pri_spec', 'assgn', 'ehr', 'mhi', 'state', 'region', 'gndr', 'prf_rate']].dropna()
mdf_cln2 = mdf_cln1[mdf_cln1.groupby('measure_id')['measure_id'].transform('count').ge(100)]
mdf_cln3 = mdf_cln2[mdf_cln2.groupby('med_sch')['med_sch'].transform('count').ge(100)]
mdf_cln4 = mdf_cln3[mdf_cln3.groupby('pri_spec')['pri_spec'].transform('count').ge(100)]
mdf_cln5 = mdf_cln4[mdf_cln3.groupby('state')['state'].transform('count').ge(100)]


In [ ]:
X_train

In [ ]:
numeric_features = ['grd_yr', 'patient_count', 'avg_num_org_mem', 'npi_num_measures']
numeric_transformer = Pipeline(steps=[
    ('poly', PolynomialFeatures()),
    ])

categorical_features = ['measure_id']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rfr', RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1))])

X = mdf_cln2.drop('prf_rate', axis=1)
y = mdf_cln2['prf_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

In [ ]:
numeric_features = ['grd_yr', 'patient_count', 'avg_num_org_mem', 'npi_num_measures']
numeric_transformer = Pipeline(steps=[
    ('poly', PolynomialFeatures()),
    ])

categorical_features = ['measure_id', 'med_sch', 'pri_spec']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rfr', RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1))])

X = mdf_cln4.drop('prf_rate', axis=1)
y = mdf_cln4['prf_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

In [ ]:
numeric_features = ['grd_yr', 'patient_count', 'avg_num_org_mem', 'npi_num_measures']
numeric_transformer = Pipeline(steps=[
    ('poly', PolynomialFeatures()),
    ])

categorical_features = ['measure_id', 'med_sch', 'pri_spec', 'assgn', 'ehr', 'mhi', 'state', 'region', 'gndr']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rfr', RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1))])

X = mdf_cln5.drop('prf_rate', axis=1)
y = mdf_cln5['prf_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

In [ ]:
numeric_features = ['grd_yr', 'patient_count', 'avg_num_org_mem', 'npi_num_measures']
numeric_transformer = Pipeline(steps=[
    ('poly', PolynomialFeatures()),
    ])

categorical_features = ['measure_id', 'med_sch', 'pri_spec', 'assgn', 'ehr', 'mhi', 'state', 'region', 'gndr']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                        ('tsvd', TruncatedSVD(n_components=200, n_iter=7, random_state=42)),
                      ('rfr', RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1))])

X = mdf_cln5.drop('prf_rate', axis=1)
y = mdf_cln5['prf_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

In [ ]:
numeric_features = ['grd_yr', 'patient_count', 'avg_num_org_mem', 'npi_num_measures']
numeric_transformer = Pipeline(steps=[
    ('rs', RobustScaler()),
    ])

categorical_features = ['measure_id', 'med_sch', 'pri_spec', 'assgn', 'ehr', 'mhi', 'state', 'region', 'gndr']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rfr', RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1))])

X = mdf_cln5.drop('prf_rate', axis=1)
y = mdf_cln5['prf_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

In [ ]:
numeric_features = ['grd_yr', 'patient_count', 'avg_num_org_mem', 'npi_num_measures']
numeric_transformer = Pipeline(steps=[
    ('rs', RobustScaler()),
    ])

categorical_features = ['measure_id', 'med_sch', 'pri_spec', 'assgn', 'ehr', 'mhi', 'state', 'region', 'gndr']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', preprocessor)
#                      ('rfr', RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1)) 
                     ])



X = mdf_cln5.drop('prf_rate', axis=1)
y = mdf_cln5['prf_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#clf.fit(X_train, y_train)
X_train = clf.fit_transform(X_train)
X_test = clf.fit_transform(X_test)

In [ ]:
mdf_fnl.describe(include='all')

In [ ]:
#missing data ndf16
total = ndf16.isnull().sum().sort_values(ascending=False)
percent = (ndf16.isnull().sum()/ndf16.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

In [ ]:
#missing data ips16
total = ips16.isnull().sum().sort_values(ascending=False)
percent = (ips16.isnull().sum()/ips16.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

In [ ]:
mdf_fnl.describe(include='all')

In [ ]:
sns.distplot(mdf_fnl.prf_rate, hist=True);
#skewness and kurtosis
print("Skewness: %f" % mdf_fnl.prf_rate.skew())
print("Kurtosis: %f" % mdf_fnl.prf_rate.kurt())

In [ ]:
#scatter plot grlivarea/saleprice
var = 'prac_yrs'
data = pd.concat([mdf_fnl['prf_rate'], mdf_fnl[var]], axis=1)
data.plot.scatter(x=var, y='prf_rate');

In [ ]:
#box plot overallqual/saleprice
var = 'pri_spec'
data = pd.concat([mdf_fnl['prf_rate'], mdf_fnl[var], mdf_fnl['gndr']], axis=1)
f, ax = plt.subplots(figsize=(100, 90))
fig = sns.boxplot(x=var, y="prf_rate", data=data, hue='gndr')
fig.axis(ymin=0, ymax=105);

In [ ]:
#correlation matrix
corrmat = mdf_fnl.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
#saleprice correlation matrix
k = 5 #number of variables for heatmap
cols = corrmat.nlargest(k, 'prf_rate')['prf_rate'].index
cm = np.corrcoef(mdf_fnl[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
describe_lvl1_measure_title = data.groupby(by=['measure_title']).describe()
describe_lvl1_med_sch = data.groupby(by=['med_sch']).describe()
describe_lvl1_gndr = data.groupby(by=['gndr']).describe()
describe_lvl1_grd_yr = data.groupby(by=['grd_yr']).describe()
describe_lvl1_pri_spec = data.groupby(by=['pri_spec']).describe()

In [ ]:
describe_lvl2_measure_title_pri_spec = data.groupby(by=['measure_title', 'pri_spec']).describe()
describe_lvl2_pri_spec_measure_title = data.groupby(by=['pri_spec', 'measure_title']).describe()


In [ ]:
#describe_lvl1_pri_spec.sort_values('')
#data.groupby(['measure_title']).filter(lambda x: x['measure_title'].count() < 100.).hist(by=data['prf_rate'], density=True)
#data.groupby(['measure_title']).hist()

grouper = data.groupby('measure_title')['prf_rate']

for days in grouper.groups.keys():
    grouper.get_group(measure_title).hist()
    plt.show()

In [ ]:
#a = data.groupby(['pri_spec', 'measure_title'])
#data.groupby(level=0).describe()
#.hist(by='', density=True, stacked=True)

data['grd_yr'].hist(by=data['measure_title'], density=True)


In [ ]:
# We name the file according to the E value, thus, we need to get all the values.
# The following line is for this setting an overkill, however, in general very nice.

# Get all unique E values 
uE = pd.Series(data.measure_title.values.ravel()).unique()
# Sort them, this is the order the hist() command returns the E value histograms        
uE.sort()

# We could use an iterator, however, we use it more often than once,
# thus, we use a counter
curE = 0   

# Go through all the histograms grouped to E
for curHist in data.groupby(['grd_yr']).hist(by='measure_title', density=True):
    # Select the figure 
    fig = curHist[0,0].get_figure()
    # Add an overall title
    fig.suptitle('Measure Title: %s' % uE[curE])
    # Make it nice
    fig.tight_layout()
    # Save the figure
    fig.savefig("../%s.png" % uE[curE], dpi=300)
    plt.close()
    
    curE += 1

In [ ]:
#describe_lvl2_measure_title_pri_spec.to_csv('../describe_lvl2_measure_title_pri_spec.csv', index=True)
describe_lvl2_measure_title_pri_spec

In [ ]:
describe_lvl1_pri_spec['prf_rate']['mean']

In [ ]:
describe_lvl1_pri_spec

In [ ]:
# Documentation of current medications
PQRS_EP_130_overallmean = PQRS_EP_130['prf_rate'].mean()
PQRS_EP_130['pri_spec_count'] = PQRS_EP_130.groupby('pri_spec')['pri_spec'].transform('count')
PQRS_EP_130['med_sch_count'] = PQRS_EP_130.groupby('med_sch')['med_sch'].transform('count')
PQRS_EP_130['gndr_count'] = PQRS_EP_130.groupby('gndr')['gndr'].transform('count')
PQRS_EP_130['pri_spec_mean'] = PQRS_EP_130.groupby('pri_spec')['prf_rate'].transform('mean')
PQRS_EP_130['med_sch_mean'] = PQRS_EP_130.groupby('med_sch')['prf_rate'].transform('mean')
PQRS_EP_130['pri_spec_mean_ratio'] = PQRS_EP_130.groupby('pri_spec')['prf_rate'].transform('mean')/PQRS_EP_130_overallmean
PQRS_EP_130['med_sch_mean_ratio'] = PQRS_EP_130.groupby('med_sch')['prf_rate'].transform('mean')/PQRS_EP_130_overallmean
#PQRS_EP_130['gndr_mean'] = PQRS_EP_130.groupby('gndr')['gndr'].transform('mean')
#PQRS_EP_130_overallmean


In [ ]:
PQRS_EP_130

In [ ]:
PQRS_EP_130[['pri_spec', 'pri_spec_count', 'pri_spec_mean', 'pri_spec_mean_ratio']].drop_duplicates().sort_values(by='pri_spec_mean_ratio')

In [ ]:
PQRS_EP_130[['med_sch', 'med_sch_count', 'med_sch_mean']].drop_duplicates().sort_values(by='med_sch_mean')

In [ ]:
# Tobacco use screening and cessation
PQRS_EP_226['pri_spec_count'] = PQRS_EP_226.groupby('pri_spec')['pri_spec'].transform('count')
PQRS_EP_226['med_sch_count'] = PQRS_EP_226.groupby('med_sch')['med_sch'].transform('count')
PQRS_EP_226['gndr_count'] = PQRS_EP_226.groupby('gndr')['gndr'].transform('count')
PQRS_EP_226['pri_spec_mean'] = PQRS_EP_226.groupby('pri_spec')['prf_rate'].transform('mean')
PQRS_EP_226['med_sch_mean'] = PQRS_EP_226.groupby('med_sch')['prf_rate'].transform('mean')



In [ ]:
PQRS_EP_226[['pri_spec', 'pri_spec_count', 'pri_spec_mean']].drop_duplicates().sort_values(by='pri_spec_mean')

In [ ]:
PQRS_EP_226[['med_sch', 'med_sch_count', 'med_sch_mean']].drop_duplicates().sort_values(by='med_sch_mean')

In [ ]:
X, y = [[thing] for thing in mdf_fnl.prac_yrs], [[thing] for thing in mdf_fnl.prf_rate]

cv = model_selection.ShuffleSplit(n_splits=20, test_size=0.2, random_state=42)
param_grid = {"knr__n_neighbors": range(100, 141, 10)}    # parameters to Pipeline take the form [label]__[estimator_param]

mdf_fnl_knn = Pipeline([('knr', KNeighborsRegressor(n_neighbors=5))
                                     ])

mdf_fnl_knn_cv = model_selection.GridSearchCV(mdf_fnl_knn, 
                                                param_grid=param_grid, cv=cv, 
                                                scoring='r2',
                                                refit=True,
                                                n_jobs=-1)
mdf_fnl_knn_cv.fit(X, y)

#param_grid = {"rfr__max_depth": (1, 2, 3, 4, 5, 6, 7)}    # parameters to Pipeline take the form [label]__[estimator_param]

#td_rfr = Pipeline([('rfr', ('rfr', RandomForestRegressor(max_depth=7, random_state=0, n_estimators=100)))
#                                     ])

#td_rfr_cv = model_selection.GridSearchCV(td_rfr, 
#                                                       param_grid=param_grid, cv=cv, 
#                                                       scoring='r2',
#                                                        refit=True)
#td_rfr_cv.fit(X, y)

In [ ]:
mdf_fnl_knn_cv_results = mdf_fnl_knn_cv.cv_results_
mdf_fnl_knn_cv_r2 = pd.DataFrame.from_dict(
    {'n_neighbors': [rec['knr__n_neighbors'] for rec in mdf_fnl_knn_cv_results['params']],
    'r2': mdf_fnl_knn_cv_results['mean_test_score']})

mdf_fnl_knn_cv_r2.r2
plt.plot(mdf_fnl_knn_cv_r2.n_neighbors, mdf_fnl_knn_cv_r2.r2)
plt.xlabel('n_neighbors')
plt.ylabel('r2')
plt.show()

In [ ]:
data = mdf[['npi', 'measure_id', 'grd_yr', 'prf_rate']].drop_duplicates().sort_values(by='measure_id')
#td = pd.concat([data[data.measure_id == 'PQRS_EP_109_1'], data[data.measure_id == 'PQRS_EP_110_1'], data[data.measure_id == 'PQRS_EP_111_1'], data[data.measure_id == 'PQRS_EP_112_1']])
td = pd.concat([data[data.measure_id == 'PQRS_EP_130_1']])

#X = mdf[['gndr', 'med_sch', 'grd_yr', 'pri_spec', 'num_org_mem']]
#y = mdf[['prf_score']]
#mdf
# features
#data
g = sns.lmplot(x='grd_yr', y='prf_rate', col='measure_id', data=td,
              col_wrap=3, ci=None, palette='muted', size=6,
              scatter_kws={"s": 5, "alpha": 1})
#g.set(ylim=(0, 100))


In [ ]:
import plotly.figure_factory as ff

figure = ff.create_scatterplotmatrix(
    td[['gndr', 'med_sch', 'grd_yr', 'pri_spec', 'prf_rate']].dropna(),
    diag='histogram')


In [ ]:
#mdf.pri_spec.unique().size
def calc_stats(df, A, B):
    print(A, B)
    return scipy.stats.describe(mdf[(mdf.measure_id == A) & (mdf.pri_spec == B)].prf_rate)
    
A = mdf.measure_id.values
B = mdf.pri_spec.values
A, B    

#df_xy['SUM_FUN0'], df_xy['SUM_FUN1'] = 
list(zip(*mdf.apply(lambda x: calc_stats(mdf, A, B), axis=1)))

#mdf[mdf.measure_id == A && mdf.pri_spec == B]
#mdf[(mdf.measure_id == 'PQRS_EP_111_1') & (mdf.pri_spec == 'INTERNAL MEDICINE')]

#mdf[(mdf.measure_id == A) & (mdf.pri_spec == B)].prf_rate


#scipy.stats.describe(mdf.prf_rate)
#print(stats.ttest_ind(c1.prf_rate,c2.prf_rate,equal_var=False))

In [ ]:
mdf

In [ ]:
grid = np.meshgrid(mdf.measure_id.values.astype(str),
                   mdf.pri_spec.values.astype(str))
#mdf.pri_spec.values.astype(str)

In [ ]:
test = pd.DataFrame({'a':range(4),'b':range(4,8)})

for a,b in zip(test.a, test.b): 
    print(a,b) 

# Calculation of PQRS "participation rate", unique providers with PQRS == 'Y' to unique providers (PQRS either 'Y' or NaN)

In [ ]:
# Calculation of PQRS "participation rate", unique providers with PQRS == 'Y' to unique providers (PQRS either 'Y' or NaN)

ndf16trunc = ndf16.loc[:, 'NPI':'pri_spec'].drop_duplicates()
ndf16trunc_pqrs = ndf16.loc[:, ['NPI', 'Ind_PAC_ID', 'lst_nm', 'frst_nm', 'Med_sch', 'Grd_yr', 'pri_spec', 'PQRS']].drop_duplicates()

pqrs_y = ndf16trunc_pqrs.loc[ndf16trunc_pqrs.PQRS == 'Y', :].PQRS.count()
pqrs_n = ndf16trunc_pqrs.PQRS.isna().count()
participation_rate_overall = pqrs_y/(pqrs_y + pqrs_n)
print(participation_rate_overall)



In [ ]:
num_practitioners_per_measure = ips16.measure_title.value_counts()

num_practitioners_per_measure.to_csv('../num_practitioners_per_measure.csv', index=True)
#measure_names_descending_frequency = ips16.measure_title.value_counts().index

#measure_names_descending_frequency_list = ips16.measure_title.value_counts().index.to_list()
#ndf16trunc.loc[:, 'pri_spec'].value_counts().to_csv('../practitoners_per_specialty.csv', index=True)
num_practitioners_per_specialty = ndf16trunc.loc[:, 'pri_spec'].value_counts()
type(num_practitioners_per_measure.to_frame())
num_practitioners_per_measure.to_frame

In [ ]:
pd.qcut(ips16.NPI, 86)
num_practitioners_per_measure.index[0]
ips16.measure_title == num_practitioners_per_measure.index[0]

In [ ]:
patient_count_per_measure = ips16.loc[:,:].groupby('measure_title').patient_count.sum().sort_values(ascending=False)


In [ ]:
num_unique_providers_in_ndf = ndf16trunc.NPI.drop_duplicates().count()
num_unique_providers_in_ips = ips16.NPI.drop_duplicates().count()
num_patients_treated = ips16.patient_count.sum()
num_measures = ips16.measure_title.drop_duplicates().count()
num_phys_specialties = ndf16trunc.pri_spec.drop_duplicates().count()

num_unique_providers_in_ndf, num_unique_providers_in_ips, num_patients_treated, num_measures, num_phys_specialties

In [ ]:
ips16.measure_title.drop_duplicates().count()

In [ ]:
## This code section examines Pneumonia Vaccination Status for Older Adults
## Need to explore weighting performance scores by patient count for 2016 data, will be interesting to contrast with the 2015 
## data where patient count is not available

a = ips16.loc[(ips16['measure_ID']=='PQRS_EP_111_1'),['NPI','prf_rate','patient_count']].drop_duplicates().dropna()

b1 = ndf16.loc[(ndf16['pri_spec']=='INTERNAL MEDICINE'),['NPI']].drop_duplicates().dropna()
c1 = pd.merge(a, b1, on='NPI')

b2 = ndf16.loc[(ndf16['pri_spec']=='INFECTIOUS DISEASE'),['NPI']].drop_duplicates().dropna()
c2 = pd.merge(a, b2, on='NPI')

b3 = ndf16.loc[(ndf16['pri_spec']=='EMERGENCY MEDICINE'),['NPI']].drop_duplicates().dropna()
c3 = pd.merge(a, b3, on='NPI')

b4 = ndf16.loc[(ndf16['pri_spec']=='PULMONARY DISEASE'),['NPI']].drop_duplicates().dropna()
c4 = pd.merge(a, b4, on='NPI')

b5 = ndf16.loc[(ndf16['pri_spec']=='NURSE PRACTITIONER'),['NPI']].drop_duplicates().dropna()
c5 = pd.merge(a, b5, on='NPI')

b6 = ndf16.loc[(ndf16['pri_spec']=='MEDICAL ONCOLOGY'),['NPI']].drop_duplicates().dropna()
c6 = pd.merge(a, b6, on='NPI')

b7 = ndf16.loc[(ndf16['pri_spec']=='GENERAL PRACTICE'),['NPI']].drop_duplicates().dropna()
c7 = pd.merge(a, b7, on='NPI')

b8 = ndf16.loc[(ndf16['pri_spec']=='PHYSICIAN ASSISTANT'),['NPI']].drop_duplicates().dropna()
c8 = pd.merge(a, b8, on='NPI')


print(scipy.stats.describe(c1.prf_rate))
print(scipy.stats.describe(c2.prf_rate))
print(scipy.stats.describe(c3.prf_rate))
print(scipy.stats.describe(c4.prf_rate))
print(scipy.stats.describe(c5.prf_rate))
print(scipy.stats.describe(c6.prf_rate))
print(scipy.stats.describe(c7.prf_rate))
print(scipy.stats.describe(c8.prf_rate))

print(stats.ttest_ind(c1.prf_rate,c2.prf_rate,equal_var=False))
print(stats.ttest_ind(c1.prf_rate,c3.prf_rate,equal_var=False))
print(stats.ttest_ind(c1.prf_rate,c4.prf_rate,equal_var=False))
print(stats.ttest_ind(c2.prf_rate,c3.prf_rate,equal_var=False))
print(stats.ttest_ind(c2.prf_rate,c4.prf_rate,equal_var=False))
print(stats.ttest_ind(c3.prf_rate,c4.prf_rate,equal_var=False))
print(stats.ttest_ind(c5.prf_rate,c8.prf_rate,equal_var=False))
print(stats.ttest_ind(c2.prf_rate,c7.prf_rate,equal_var=False))
print(stats.ttest_ind(c1.prf_rate,c5.prf_rate,equal_var=False))
print(stats.ttest_ind(c1.prf_rate,c8.prf_rate,equal_var=False))
print(1-58.85706462212487/75.30104623692203)
print(1-59.421996879875195/75.30104623692203)
print(1-62.54601226993865/75.30104623692203)
print(1-63.55072463768116/75.30104623692203)

In [ ]:
phys_specialties = ndf16t.pri_spec.drop_duplicates()
med_schools = ndf16t.Med_sch.drop_duplicates()
measure_num_name_pairs = ips16[['measure_ID', 'measure_title']].drop_duplicates()
measure_num_name_pairs

#med_schools.drop_duplicates().to_csv('../med_sch.csv', index=False)
#measure_num_name_pairs.drop_duplicates().to_csv('../measuretitles.csv', index=False)
#phys_specialties.drop_duplicates().to_csv('../phys_specialties.csv', index=False)

#phys_specialties
#ips16[ips16['NPI','measure_ID']]

# Figure 1

In [ ]:
dy = np.linspace(1945,2020,16)
a16 = pd.value_counts(ndf16.loc[(ndf16['PQRS']=='Y'),['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
b16 = pd.value_counts(ndf16[['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
c16 = a16/b16
ax = c16.plot.bar(ylim=(0,0.7), title='Clinician Participation Rate vs Year of Terminal Degree (2016 Data)')
ax.set(xlabel="Year of Clinician's Terminal Degree", ylabel="Clinician participation rate")

# Figure 2

In [ ]:
a16 = pd.value_counts(ndf16.loc[(ndf16['PQRS']=='Y'),['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
b16 = pd.value_counts(ndf16[['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
c16 = a16/b16
ax = c16.plot.bar(ylim=(0,0.7), title='Clinician Participation Rate vs Year of Terminal Degree (2016 Data)')
ax.set(xlabel="Year of Clinician's Terminal Degree", ylabel="Clinician participation rate")

# Figure 3

In [ ]:
ax = (c16/c15).iloc[0:13].plot.bar(ylim=(0.9,1.1), title='Participation rate change vs year of terminal degree (2016 data vs 2015 data)')
ax.set(xlabel="Year of Clinician's Terminal Degree", ylabel="Clinician participation rate year over year change")
#print((c16/c15).iloc[0:13])

# Pneumonia Results

# It has been estimated that the total annual excess cost of hospital-treated pneumonia as a primary diagnosis in the elderly fee-for-service Medicare population in 2010 exceeded 7 billion dollars.

# Takeaways from the above cursory comparisons of how clinicians of one primary specialty perform relative to clinicians of another primary specialty regarding how they advise their patients on Pneumonia Vaccination. 

# Physician Assistants (PAs) and Nurse Practitioners (NPs) do similarly well (p = 0.57) but both do significantly poorer than those whose primary specialty was Internal Medicine, with mean score 20% lower

# Surprisingly, those whose primary specialty was Infectious Disease did significantly worse than Internal Medicine (p = 3e-08, Infectious Disease mean score 17% lower) and  Pulmonary Disease (p = 2e-05, Infectious Disease mean score 16% lower), but similarly to General Medicine (p = 0.8).


In [ ]:
female = df1.loc[(df1["Gender"]=="F"), ["Gender"]]
female_accept = df1.loc[(df1["Gender"]=="F") & (df1["Professional accepts Medicare Assignment"]=="Y"), ["NPI","PAC ID","Medical school name","Graduation year"]]
male = df1.loc[(df1["Gender"]=="M"), ["Gender"]]
male_accept = df1.loc[(df1["Gender"]=="M") & (df1["Professional accepts Medicare Assignment"]=="Y"), ["NPI","PAC ID","Medical school name","Graduation year"]]
am = df1.loc[(df1["Gender"]=="M") & (df1["Professional accepts Medicare Assignment"]=="Y"), ["NPI","Medical school name","Graduation year"]]
af = df1.loc[(df1["Gender"]=="F") & (df1["Professional accepts Medicare Assignment"]=="Y"), ["NPI","Medical school name","Graduation year"]]
df2_flu = df2.loc[(df2["Measure Title"]=="Preventive Care and Screening: Influenza Immunization"),["NPI","Measure Performance Rate"]]
df3_pneuvacc = df2.loc[(df2["Measure Title"]=="Pneumonia Vaccination Status for Older Adults"),["NPI","Measure Performance Rate"]]
df4_brcanc = df2.loc[(df2["Measure Title"]=="Breast Cancer Screening"),["NPI","Measure Performance Rate"]]
df5_colocanc = df2.loc[(df2["Measure Title"]=="Colorectal Cancer Screening"),["NPI","Measure Performance Rate"]]
df6_bmi = df2.loc[(df2["Measure Title"]=="Preventive Care and Screening: Body Mass Index (BMI) Screening and Follow-Up Plan"),["NPI","Measure Performance Rate"]]
df7_currmeds = df2.loc[(df2["Measure Title"]=="Documentation of Current Medications in the Medical Record"),["NPI","Measure Performance Rate"]]
df8_eldmal = df2.loc[(df2["Measure Title"]=="Elder Maltreatment Screen and Follow-Up Plan"),["NPI","Measure Performance Rate"]]
df9_tobacco = df2.loc[(df2["Measure Title"]=="Preventive Care and Screening: Tobacco Use: Screening and Cessation Intervention"),["NPI","Measure Performance Rate"]]
df10_osteo = df2.loc[(df2["Measure Title"]=="Screening or Therapy for Osteoporosis for Women Aged 65 Years and Older"),["NPI","Measure Performance Rate"]]
df11_ecg = df2.loc[(df2["Measure Title"]=="Emergency Medicine: 12-Lead Electrocardiogram (ECG) Performed for Non-Traumatic Chest Pain"),["NPI","Measure Performance Rate"]]

brcancm=pd.merge(am, df4_brcanc, on='NPI').drop_duplicates().dropna()
brcancf=pd.merge(af, df4_brcanc, on='NPI').drop_duplicates().dropna()



In [ ]:
colocanc_tobacco=pd.merge(ips15.loc[(ips15['measure_title']=='Colorectal Cancer Screening'),['NPI','prf_rate']],ips15.loc[(ips15['measure_title']=='Preventive Care and Screening: Tobacco Use: Screening and Cessation Intervention'),['NPI','prf_rate']], on='NPI').drop_duplicates().dropna()
flu_pneuvacc=pd.merge(ips15.loc[(ips15['measure_title']=='Preventive Care and Screening: Influenza Immunization'),['NPI','prf_rate']],ips15.loc[(ips15['measure_title']=='Pneumonia Vaccination Status for Older Adults'),['NPI','prf_rate']], on='NPI').drop_duplicates().dropna()
brcancm = pd.merge(ips15.loc[(ips15['measure_title']=='Breast Cancer Screening'),['NPI','prf_rate']],ndf15.loc[(ndf15.gndr=='M'),['NPI','Grd_yr']],on='NPI').drop_duplicates().dropna()
brcancf = pd.merge(ips15.loc[(ips15['measure_title']=='Breast Cancer Screening'),['NPI','prf_rate']],ndf15.loc[(ndf15.gndr=='F'),['NPI','Grd_yr']],on='NPI').drop_duplicates().dropna()
flum = pd.merge(ips15.loc[(ips15['measure_title']=='Preventive Care and Screening: Influenza Immunization'),['NPI','prf_rate']],ndf15.loc[(ndf15.gndr=='M'),['NPI','Grd_yr']],on='NPI').drop_duplicates().dropna()
fluf = pd.merge(ips15.loc[(ips15['measure_title']=='Preventive Care and Screening: Influenza Immunization'),['NPI','prf_rate']],ndf15.loc[(ndf15.gndr=='F'),['NPI','Grd_yr']],on='NPI').drop_duplicates().dropna()

# Figure 4

In [ ]:
ax = colocanc_tobacco.plot.scatter(x='prf_rate_x',y='prf_rate_y',title='Coincidence of performance scores in Colon Cancer Prevention and Smoking Intervention')
ax.set(xlabel='Colon Cancer Screening performance rate', ylabel='Smoking Intervention')
print(scipy.stats.describe(colocanc_tobacco.prf_rate_x))
print(scipy.stats.describe(colocanc_tobacco.prf_rate_y))
print(stats.ttest_ind(colocanc_tobacco.prf_rate_x,colocanc_tobacco.prf_rate_y,equal_var=False))

# Statistical analysis confirms the surprising result seen above. Clinicians who had scores in both categories did an excellent job of counseling their patients with regards to Smoking Cessation (mean value ~95), but those same clinicians did much more poorly when it came to Colorectal Cancer Screening (mean value ~56). Colorectal cancers affect mosly older adults, and the prognosis of those suffering from such cancers drops sharply with disease progression. 

# Figure 5

In [ ]:
fig=plt.figure()
fig.suptitle('Clinician Performance Score for Breast Cancer Screening', fontsize=20)
plt.xlabel('Clinician Graduation Year')
plt.ylabel('Performance Score')
ax1=plt.scatter(brcancm.Grd_yr, brcancm.prf_rate, c='blue', s=0.4, label='Male')
ax2=plt.scatter(brcancf.Grd_yr, brcancf.prf_rate, c='red', s=0.4, label='Female')
plt.gca().legend()
plt.show()

# Figure 6

In [ ]:
fig=plt.figure()
fig.suptitle('Clinician Performance Score for Breast Cancer Screening',fontsize=20)
plt.xlabel('Performance Score for Breast Cancer Screening')
plt.ylabel('Normalized Frequency')
ax1=plt.hist(brcancm.prf_rate, label="Male", bins=dp, color='blue', normed=True)
ax2=plt.hist(brcancf.prf_rate, label="Female", bins=dp, color='red', normed=True, alpha=0.25)
plt.gca().legend()
plt.show()
print(stats.ttest_ind(brcancm.prf_rate,brcancf.prf_rate,equal_var=False))
print(scipy.stats.describe(brcancm.prf_rate))
print(scipy.stats.describe(brcancf.prf_rate))
print(1-57.528294367693945/63.06273836765828)

# Female clinicians perform significantly (p = 1e-31) better at Breast Cancer Screening than do their Male counterparts, with the Male mean score ~9% lower than that of the Female mean score. 

# Figure 7

In [ ]:
fig=plt.figure()
fig.suptitle('Clinician Performance Score for Influenza Administration', fontsize=20)
plt.xlabel('Clinician Graduation Year')
plt.ylabel('Performance Score')
ax1=plt.scatter(flum.Grd_yr, flum.prf_rate, c='blue', s=0.4, label='Male')
ax2=plt.scatter(fluf.Grd_yr, fluf.prf_rate, c='red', s=0.4, label='Female')
plt.gca().legend()
plt.show()

# Figure 8

In [ ]:
fig=plt.figure()
fig.suptitle('Clinician Performance Score for Influenza Immunization',fontsize=20)
plt.xlabel('Score for Influenza Immunization')
plt.ylabel('Normalized Frequency')
ax1=plt.hist(flum.prf_rate, label="Male", bins=dp, color='blue', normed=True)
ax2=plt.hist(fluf.prf_rate, label="Female", bins=dp, color='red', normed=True, alpha=0.25)
plt.gca().legend()
plt.show()
print(stats.ttest_ind(flum.prf_rate,fluf.prf_rate,equal_var=False))
print(scipy.stats.describe(flum.prf_rate))
print(scipy.stats.describe(fluf.prf_rate))
print(1-scipy.stats.describe(flum.prf_rate).mean/scipy.stats.describe(fluf.prf_rate).mean)

# Female clinicians perform significantly (p = 7e-08) better at Influenza Immunization than do their Male counterparts, with the Male mean score ~5% lower than that of the Female mean score.

In [ ]:
dy = np.linspace(1945,2020,16)
a15 = pd.value_counts(ndf15.loc[(ndf15['PQRS']=='Y'),['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
b15 = pd.value_counts(ndf15[['NPI','Grd_yr']].drop_duplicates().dropna().Grd_yr, bins=dy, sort=False)
c15 = a15/b15
ax = c15.plot.bar(ylim=(0,0.7), title='Clinician Participation Rate vs Year of Terminal Degree (2015 Data)')
ax.set(xlabel="Year of Clinician's Terminal Degree", ylabel="Clinician participation rate")

# Figure 9

In [ ]:
ax = flu_pneuvacc.plot.scatter(x='prf_rate_x',y='prf_rate_y',title='Coincidence of performance scores in Influenza Immunization and Pneumonia Vaccination')
ax.set(xlabel='Influenza Immunization Score', ylabel='Pneumonia Vaccination Score')
print(scipy.stats.describe(flu_pneuvacc.prf_rate_x))
print(scipy.stats.describe(flu_pneuvacc.prf_rate_y))
print(stats.ttest_ind(flu_pneuvacc.prf_rate_x,flu_pneuvacc.prf_rate_y,equal_var=False))
print(1-scipy.stats.describe(flu_pneuvacc.prf_rate_x).mean/scipy.stats.describe(flu_pneuvacc.prf_rate_y).mean)

# On average clinicians do a significantly worse (p = 4e-237) job counseling their patients regarding Influenza Vaccination (mean score = 50.8, 18% lower than for Pneumonia Vaccination) than they do regarding Pneumonia Vaccination (mean score = 61.8). 

# In Progress, need to vectorize loops(!) and extend into the measure_ID dimension

In [ ]:
## This section collects descriptive statistics on the performance data of clinicians by their primary specialty, generating a
## dataframe in which the columns are specialties and the columns are the various measurement categories covered by the PQRS
## As of now this works on only a single category, and compares clinician primary specialties within that category.
## This needs to be vectorized badly, extremely slow as nested for loops.

spec_list = ndf15.pri_spec.unique()
measure_ID_list = ips15.measure_ID.unique()
spec_mid_descstat = pd.DataFrame(index=np.arange(0, len(spec_list)), columns=measure_ID_list)

for i in range(0, len(measure_ID_list)):
    a = ips15.loc[(ips15['measure_ID']==measure_ID_list[i]),['NPI', 'prf_rate', 'patient_count']].drop_duplicates().dropna()
    for j in range(0, len(spec_list)):
        b = ndf15.loc[(ndf15['pri_spec']==spec_list[j]),['NPI']].drop_duplicates().dropna()
        c = pd.merge(a, b, on='NPI')
        if not c.prf_rate.empty:
            d = scipy.stats.describe(c.prf_rate)
            e = c.patient_count.sum()
            spec_mid_descstat.iat[j,i] = d


In [ ]:
## Ttests between specialties for a given performance measure

spec_mid_ttests = pd.DataFrame(index=spec_list, columns=spec_list)

for i in range(0, len(spec_list)):
    for j in range(0, len(spec_list)):
        if not pd.isnull(spec_mid_descstat.iat[i,0])|pd.isnull(spec_mid_descstat.iat[j,0]) :
            d = scipy.stats.ttest_ind_from_stats(mean1=spec_mid_descstat.iat[i,0].mean, std1=np.sqrt(spec_mid_descstat.iat[i,0].variance), nobs1=spec_mid_descstat.iat[i,0].nobs, mean2=spec_mid_descstat.iat[j,0].mean, std2=np.sqrt(spec_mid_descstat.iat[j,0].variance), nobs2=spec_mid_descstat.iat[j,0].nobs, equal_var=False)
            spec_mid_ttests.iat[j,i] = d
        else:
            spec_mid_ttests.iat[j,i] = d

In [ ]:
#spec_mid_ttests
# Construct heatmap using pvalues?
spec_mid_ttests.iat[3,0].pvalue